In [ ]:
%pip install tensorflow
%pip install tensorflow-hub
%pip install tensorflow-datasets
%pip install tensorflow-lite

In [ ]:
%pip uninstall tensorflow --no-cache-dir

In [ ]:
%pip install numpy==1.19.2
%pip install protobuf==3.20.3
%pip install daal==2021.4.0
%pip install clyent==1.2.1
%pip install nbformat==5.4.0
%pip install typing-extensions==3.10.0.2
%pip install tensorflow==2.6.0


In [4]:
import tensorflow as tf
# Clear the session
tf.keras.backend.clear_session()

print(f'testing')
# Load the Keras models
classification_model = tf.keras.models.load_model('C:\\Users\\thiru\\Desktop\\Aashritha project\\python_code\\New_24_03_TTC_classification_model.h5')
regression_model = tf.keras.models.load_model('C:\\Users\\thiru\\Desktop\\Aashritha project\\python_code\\TTC_regression_model.h5')

# Convert the classification model to a TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_keras_model(classification_model)
tflite_classification_model = converter.convert()

# Convert the regression model to a TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_keras_model(regression_model)
tflite_regression_model = converter.convert()


INFO:tensorflow:Assets written to: C:\Users\thiru\AppData\Local\Temp\tmpiu51r0_b\assets
INFO:tensorflow:Assets written to: C:\Users\thiru\AppData\Local\Temp\tmp9s807n6t\assets


INFO:tensorflow:Assets written to: C:\Users\thiru\AppData\Local\Temp\tmp9s807n6t\assets


In [5]:
import os
model_folder_path = "C:\\Users\\thiru\\Desktop\\Aashritha project\\python_code"
os.makedirs(model_folder_path, exist_ok=True)

# Save the classification model to disk
classification_model_path = os.path.join(model_folder_path, "classification_model.tflite")
with open(classification_model_path, "wb") as f:
    f.write(tflite_classification_model)

# Save the regression model to disk
regression_model_path = os.path.join(model_folder_path, "regression_model.tflite")
with open(regression_model_path, "wb") as f:
    f.write(tflite_regression_model)


In [6]:
# Define the input and output signatures for the combined model
classification_input = classification_model.inputs[0]
regression_input = regression_model.inputs[0]
classification_output = classification_model.outputs[0]
regression_output = regression_model.outputs[0]
combined_input = tf.TensorSpec(shape=(1, 4), dtype=tf.float32)
classification_decision = tf.TensorSpec(shape=(1,), dtype=tf.int64)
final_output = tf.TensorSpec(shape=(1,), dtype=tf.float32)

In [7]:
# Define the TensorFlow function to be converted to a TensorFlow Lite model
@tf.function(input_signature=[combined_input])
def predict_tflite(inputs):
    # Run the classification model on the inputs
    classification_result = classification_model(inputs)
    # Get the classification decision
    classification_decision_result = tf.argmax(classification_result, axis=1)
    # Run the regression model on the inputs
    regression_result = regression_model(inputs)
    # Combine the outputs based on the classification decision
    final_result = tf.cond(tf.equal(classification_decision_result, 1),
                           lambda: regression_result,
                           lambda: classification_result)
    return final_result


In [9]:
# Convert the TensorFlow function to a TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_concrete_functions(
    [predict_tflite.get_concrete_function(combined_input)],
    trackable_obj=predict_tflite)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

In [10]:
# Create a Checkpoint object to track the state of the function
checkpoint = tf.train.Checkpoint(predict=predict_tflite)
checkpoint.save('/tmp/ckpt/')

# Convert the TensorFlow function to a TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_concrete_functions([predict_tflite.get_concrete_function(combined_input)], trackable_obj=checkpoint)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()


In [ ]:
# Save the TensorFlow Lite model to a file
regression_model_path = os.path.join(model_folder_path, "regression_model.tflite")
with open(regression_model_path, "wb") as f:
with open('combined_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
import numpy as np

# Load the TensorFlow Lite models
interpreter_classification = tf.lite.Interpreter(model_content=tflite_classification_model)
interpreter_classification.allocate_tensors()

interpreter_regression = tf.lite.Interpreter(model_content=tflite_regression_model)
interpreter_regression.allocate_tensors()

# Prepare the input data
input_data = np.random.rand(1, 4)  # replace input_shape with the actual input shape of your models

# Set the input tensor values
input_details_classification = interpreter_classification.get_input_details()
interpreter_classification.set_tensor(input_details_classification[0]['index'], input_data)

input_details_regression = interpreter_regression.get_input_details()
interpreter_regression.set_tensor(input_details_regression[0]['index'], input_data)

# Invoke the interpreter
interpreter_classification.invoke()
interpreter_regression.invoke()

# Get the output tensor values
output_details_classification = interpreter_classification.get_output_details()
classification_result = interpreter_classification.get_tensor(output_details_classification[0]['index'])

output_details_regression = interpreter_regression.get_output_details()
regression_result = interpreter_regression.get_tensor(output_details_regression[0]['index'])


In [2]:
import os
print(os.getcwd())

d:\github_workspace\Project_2023\vehicle-collision-avoidance-ann\TensorflowTTCModel
